IAMSI -- 2018-2019
--------
*&copy; Equipe pédagogique : Gauvain Bourgne, Jean-Gabriel Ganascia, Colette Faucher, Marie-Jeanne Lesot, Christophe Marsala.*

# TME 09 : Règles d'association

<font color="RED" size="+1">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms **

Lam Jean-Paul, Souksava Elodie

<font color="RED" size="+1">**[Q]**</font> **Renommer ce noebook**

Tout en haut de cette page, cliquer sur <tt>TME-09</tt> et rajouter à la suite de <tt>TME-09</tt> les noms des membres du binômes séparés par un tiret.

Par exemple, pour le binôme Luke Skywalker et Han Solo, le nom de fichier devient : <pre>TME-09-Skywalker-Solo</pre>

Penser à sauvegarder fréquemment le fichier en cours de travail :
- soit en cliquant sur l'icône "disquette"
- soit par la combinaison de touches [Ctrl]-S

## Présentation

### But de ce TME

Ce TME a pour but de réaliser une implémentation *intuitive* de l'algorithme **apriori** afin de la comparer à une version efficace de cet algorithme, puis à une implémentation de l'algorithme **fp-growth**.


#### Contenu de l'archive téléchargée

Une fois détarée, l'archive **tme-09.tgz** crée le répertoire <code>tme09</code> contenant :
- deux fichiers exécutables (Linux): <code>apriori32</code> et <code>fpgrowth32</code> (ces 2 programmes ont été développés par Christian Borgelt, plus d'infos sur sa page web  http://www.borgelt.net/software.html). Ces programmes seront utilisés en fin de séance.
- un répertoire <code>dataset</code> contenant des bases d'exemples pour tester les algorithmes
- ce notebook que vous lisez en ce moment et que vous allez compléter.



#### Compte-rendu de la séance

Le compte-rendu de ce TME se compose de ce notebook complété par les réponses aux questions posées.

Ce compte-rendu est à poster : 
- à l'issue de la séance, un premier envoi doit être **obligatoirement** fait avec ce que vous avez réalisé
- si nécessaire, un complément peut être envoyé **au plus tard avant le début de la prochaine séance**.


<font color="RED">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: envoyer le notebook tel quel, éventuellement, si vous avez d'autres fichiers à envoyer, les  joindre au message.

In [1]:
# Vérification de la version de Python utilisée:
import sys
sys.path # le path doit contenir python3.5

['C:\\Users\\Jp',
 'C:\\Users\\Jp\\AppData\\Local\\Continuum\\anaconda3\\python37.zip',
 'C:\\Users\\Jp\\AppData\\Local\\Continuum\\anaconda3\\DLLs',
 'C:\\Users\\Jp\\AppData\\Local\\Continuum\\anaconda3\\lib',
 'C:\\Users\\Jp\\AppData\\Local\\Continuum\\anaconda3',
 '',
 'C:\\Users\\Jp\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages',
 'C:\\Users\\Jp\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\Jp\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Jp\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Jp\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Jp\\.ipython']

## Traitement d'une base d'apprentissage

### Chargement de la base

On commence par travailler sur la base exemple du fichier <code>exemple-1.txt</code> (fourni dans le répertoire datasets). 

Ce fichier contient une transaction par ligne. Chaque transaction est composée d'un groupe d'items séparés par un espace.

On peut charger en Python ce fichier par la commande suivante (le répertoire datasets doit se trouver dans le répertoire courant) :

In [2]:
import csv
with open('datasets/exemple-1.txt', 'r') as fichier:
    lecteur = csv.reader(fichier, delimiter=' ')
    i = 0
    for ligne in lecteur:
        i += 1
        print('ligne',i,':',ligne)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/exemple-1.txt'

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>chargeBase</code> qui prend en argument un nom de fichier, respectant le format énoncé plus haut, le lit et rend un dictionnaire dont les clés sont les numéros de ligne (une transaction) et les valeurs associées les itemsets correspondants représentés sous forme d'ensembles Python (des <code>set()</code> donc).

Ici, il est plus intéressant de représenter un itemset comme un ensemble Python plutôt que comme une liste. Il est ainsi plus facile de réaliser des comparaisons d'ensembles ou des ajouts d'éléments.

Dans le reste de ce document, on appelle **BASE** un dictionnaire de ce type.

In [58]:
def chargeBase(fichier,delimiter=' '):
    d = {}
    with open(fichier, 'r') as f:
        lecteur = csv.reader(f, delimiter=delimiter)
        i = 0
        for ligne in lecteur:
            i += 1
            d[i] = set(ligne)
        #print(d)
    return d

In [59]:
print("Résultat du chargement de 'exemple-1.txt', on obtient : ")
Base1 = chargeBase('datasets/exemple-1.txt',delimiter=' ')
print(Base1)

Résultat du chargement de 'exemple-1.txt', on obtient : 
{1: {'c', 'b', 'a'}, 2: {'e', 'd', 'a'}, 3: {'c', 'b', 'd'}, 4: {'c', 'b', 'd', 'a'}, 5: {'c', 'b'}, 6: {'b', 'd', 'a'}, 7: {'e', 'd'}, 8: {'c', 'b', 'd', 'a'}, 9: {'e', 'c', 'd'}, 10: {'c', 'b', 'a'}}


On utilise la variable **Base1** dans la suite pour faire référence à cette base.

## Itemsets et support

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>noms_items</code> qui prend en argument une BASE et rend l'ensemble des items qui composent cette base.

In [6]:
def noms_items(base):
    item=base[1]
    for i in range(2,len(base)+1) : 
        item=item.union(base[i])
   # print(item)
    return item

In [7]:
print("Pour la BASE précédente :")
noms_items(Base1)


Pour la BASE précédente :


{'a', 'b', 'c', 'd', 'e'}

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>singletons</code> qui prend en argument une BASE et rend la liste des itemsets de taille 1 obtenus à partir de cette base.

*Remarque*: attention, ici on utilise une **liste** pour stocker les itemsets (un itemset est un ensemble Python) car il n'est pas possible en Python de créer des ensembles d'ensembles.


In [8]:
def singletons(base):
    item=noms_items(base)
    s=[]
    for i in item:
        s.append({i})
    return s

In [9]:
print("Exemple: singletons(Base1) rend :")
singletons(Base1)

Exemple: singletons(Base1) rend :


[{'d'}, {'e'}, {'a'}, {'b'}, {'c'}]

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>comptage</code> qui, pour une BASE et un itemset donnés, rend le nombre de transactions de BASE qui contiennent cet itemset.

In [10]:
def comptage(base,itemset):
    cpt=0
    for i in base:
        cpt+=itemset<=base[i]
    return cpt

In [11]:
print("Comptage de l'itemset {'a','b','c'} dans la base précédente : ")
print("comptage(Base1,{'a','b','c'}) rend la valeur "+str(comptage(Base1,{'a','b','c'})))

Comptage de l'itemset {'a','b','c'} dans la base précédente : 
comptage(Base1,{'a','b','c'}) rend la valeur 4


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>support</code> qui, pour une BASE et un itemset donnés, rend le support de cet itemset dans la BASE.

In [12]:
def support(base,itemset):
    return comptage(base,itemset)/len(base)

In [13]:
print("Support de l'itemset {'a','b','c'} dans la base précédente : " \
      +str(support(Base1,{'a','b','c'})))

Support de l'itemset {'a','b','c'} dans la base précédente : 0.4


## Implémentation de l'algorithme a-priori

Dans cette partie, une implémentation de la partie de construction des itemsets fréquents de l'algorithme a-priori est réalisée. On ne s'intéresse pas dans cette question à la génération des règles d'association (mais cela peut être fait en complément).

Votre programme doit pouvoir s'appliquer aux bases fournies dans le répertoire *datasets* (éventuellement, sur au moins les 10 premiers exemples de la base mushrooms).

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>apriori_gen</code> qui prend en argument une liste d'itemsets de même longueur $k$, applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats de longueurs $k+1$.


In [75]:
import copy
def apriori_gen(F):
    if len(F)==0:
        return []
    k = len(F[0])
    c={}
    C=[]
    for i in range(len(F)-1):
        for j in range(i+1,len(F)):
            c=F[i].union(F[j])
            if(len(c)==k+1 and c not in C):
                C.append(c)
    liste_enleve = []
    for c in C:
        for i in c:
            c2 = copy.deepcopy(c)
            c2.remove(i)
            if c2 not in F and c not in liste_enleve:
                liste_enleve.append(c)
    for c in liste_enleve:
        C.remove(c)
    return C

In [15]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]))

print("\nExemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend ")
print(apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]))

Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'b', 'a'}, {'c', 'a'}, {'d', 'a'}, {'c', 'b'}, {'b', 'd'}, {'c', 'd'}]

Exemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend 
[{'b', 'd', 'a'}, {'c', 'b', 'd'}]


In [16]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]))

print("\nExemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend ")
print(apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]))

Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'b', 'a'}, {'c', 'a'}, {'d', 'a'}, {'c', 'b'}, {'b', 'd'}, {'c', 'd'}]

Exemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend 
[{'b', 'd', 'a'}, {'c', 'b', 'd'}]


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>apriori</code> qui prend en argument une BASE et une valeur réelle comprise entre 0 et 1, et qui rend une liste de tuples dont le premier élément et un itemset trouvé et le deuxième élément est la valeur de support correspondante.

In [76]:
def apriori(base,p):
    Ck=singletons(base)
    k=1
    F=[]
    while(Ck!=[]):
        Fk=[]
        for c in Ck:
            sup=support(base,c)
            if (sup>=p):
                Fk.append(c)
                F.append((c,sup))
        k+=1
        Ck=apriori_gen(Fk)
        
    return F

In [16]:
#verification
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'a'}, 0.6), ({'d'}, 0.7), ({'c'}, 0.7), ({'e'}, 0.3), ({'b'}, 0.7), ({'a', 'd'}, 0.4), ({'a', 'c'}, 0.4), ({'a', 'b'}, 0.5), ({'d', 'c'}, 0.4), ({'d', 'e'}, 0.3), ({'b', 'd'}, 0.4), ({'b', 'c'}, 0.6), ({'a', 'b', 'd'}, 0.3), ({'a', 'b', 'c'}, 0.4), ({'b', 'd', 'c'}, 0.3)]


In [20]:
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'d'}, 0.7), ({'e'}, 0.3), ({'a'}, 0.6), ({'b'}, 0.7), ({'c'}, 0.7), ({'e', 'd'}, 0.3), ({'d', 'a'}, 0.4), ({'b', 'd'}, 0.4), ({'c', 'd'}, 0.4), ({'b', 'a'}, 0.5), ({'c', 'a'}, 0.4), ({'c', 'b'}, 0.6), ({'b', 'd', 'a'}, 0.3), ({'c', 'b', 'd'}, 0.3), ({'c', 'b', 'a'}, 0.4)]


## Expérimentations

### Vérification avec la base du TD

Utiliser la fonction <code>apriori</code> avec la base de transactions du TD afin de vérifier que les itemsets obtenus sont corrects.

In [23]:
BaseTD={1:{'a','b','d','e'},2:{'b','c','d'},3:{'a','b','d','e'},4:{'a','c','d','e'},5:{'b','c','d','e'},6:{'b','d','e'},7:{'c','d'},8:{'a','b','c'},9:{'a','d','e'},10:{'b','d'}}

In [20]:
#verification
print("Exemple: apriori(BaseTD, 0.3) rend ")
print(apriori(BaseTD,0.3))

Exemple: apriori(BaseTD, 0.3) rend 
[({'a'}, 0.5), ({'d'}, 0.9), ({'e'}, 0.6), ({'c'}, 0.5), ({'b'}, 0.7), ({'a', 'd'}, 0.4), ({'a', 'e'}, 0.4), ({'a', 'b'}, 0.3), ({'d', 'e'}, 0.6), ({'d', 'c'}, 0.4), ({'b', 'd'}, 0.6), ({'b', 'e'}, 0.4), ({'b', 'c'}, 0.3), ({'a', 'd', 'e'}, 0.4), ({'b', 'd', 'e'}, 0.4)]


In [46]:
print("Exemple: apriori(BaseTD, 0.3) rend ")
print(apriori(BaseTD,0.3))

Exemple: apriori(BaseTD, 0.3) rend 
[({'b'}, 0.7), ({'d'}, 0.9), ({'e'}, 0.6), ({'c'}, 0.5), ({'a'}, 0.5), ({'b', 'd'}, 0.6), ({'e', 'b'}, 0.4), ({'c', 'b'}, 0.3), ({'b', 'a'}, 0.3), ({'e', 'd'}, 0.6), ({'c', 'd'}, 0.4), ({'d', 'a'}, 0.4), ({'e', 'a'}, 0.4), ({'e', 'b', 'd'}, 0.4), ({'e', 'd', 'a'}, 0.4)]


### Comparaisons avec les implémentations apriori32 et fpgrowth

<font color="RED" size="+1">**[Q]**</font>  Tester votre fonction <code>apriori</code> sur les 3 bases données dans datasets et confronter les résultats obtenus avec ceux fournis par le programme <code>apriori32</code>.

In [61]:
print("Résultat du chargement de 'mushrooms.txt', on obtient : ")
Base_mushrooms = chargeBase('datasets/mushrooms.txt',delimiter=' ')
#Base_mushrooms
print(len(Base_mushrooms),' transactions.')

Résultat du chargement de 'mushrooms.txt', on obtient : 
8416  transactions.


In [51]:
print("Pour la BASE précédente :")
noms_items(Base_mushrooms)
print("Exemple: singletons(Base_mushrooms) rend :")
singletons(Base_mushrooms)

Pour la BASE précédente :
Exemple: singletons(Base_mushrooms) rend :


[{'PINK'},
 {'ONE'},
 {'CLUB'},
 {'CLOSE'},
 {'?'},
 {'SMOOTH'},
 {'FREE'},
 {'CONVEX'},
 {'FLARING'},
 {'SPICY'},
 {'URBAN'},
 {'BUFF'},
 {'GRASSES'},
 {'ROOTED'},
 {'GROOVES'},
 {'FISHY'},
 {'FOUL'},
 {'NARROW'},
 {'BROWN'},
 {'BELL'},
 {'PURPLE'},
 {'KNOBBED'},
 {'ENLARGING'},
 {'RED'},
 {'EVANESCENT'},
 {'SOLITARY'},
 {'PARTIAL'},
 {'PUNGENT'},
 {'NUMEROUS'},
 {'MEADOWS'},
 {'GREEN'},
 {'SUNKEN'},
 {'ORANGE'},
 {'NO'},
 {'EQUAL'},
 {'TWO'},
 {'BULBOUS'},
 {'FLAT'},
 {'CINNAMON'},
 {'WHITE'},
 {'LARGE'},
 {'WASTE'},
 {'LEAVES'},
 {'PENDANT'},
 {'SILKY'},
 {'PATHS'},
 {'BROAD'},
 {'CHOCOLATE'},
 {'ALMOND'},
 {'SCALY'},
 {'MUSTY'},
 {'ANISE'},
 {'SEVERAL'},
 {'FIBROUS'},
 {'TAPERING'},
 {'NONE'},
 {'BRUISES'},
 {'BLACK'},
 {'ATTACHED'},
 {'CROWDED'},
 {'SCATTERED'},
 {'GRAY'},
 {'CREOSOTE'},
 {'CONICAL'},
 {'YELLOW'},
 {'WOODS'},
 {'CLUSTERED'},
 {'ABUNDANT'}]

In [23]:
#VERIFICATION
print(apriori(Base_mushrooms,0.9))

[({'PARTIAL'}, 1.0), ({'WHITE'}, 0.9771863117870723), ({'ONE'}, 0.9230038022813688), ({'FREE'}, 0.9743346007604563), ({'PARTIAL', 'WHITE'}, 0.9771863117870723), ({'PARTIAL', 'ONE'}, 0.9230038022813688), ({'PARTIAL', 'FREE'}, 0.9743346007604563), ({'ONE', 'WHITE'}, 0.9001901140684411), ({'FREE', 'WHITE'}, 0.9743346007604563), ({'FREE', 'ONE'}, 0.9001901140684411), ({'PARTIAL', 'ONE', 'WHITE'}, 0.9001901140684411), ({'PARTIAL', 'FREE', 'WHITE'}, 0.9743346007604563), ({'PARTIAL', 'ONE', 'FREE'}, 0.9001901140684411), ({'FREE', 'ONE', 'WHITE'}, 0.9001901140684411), ({'FREE', 'PARTIAL', 'ONE', 'WHITE'}, 0.9001901140684411)]


In [52]:
print(apriori(Base_mushrooms,0.9))

[({'ONE'}, 0.9230038022813688), ({'FREE'}, 0.9743346007604563), ({'PARTIAL'}, 1.0), ({'WHITE'}, 0.9771863117870723), ({'ONE', 'FREE'}, 0.9001901140684411), ({'ONE', 'PARTIAL'}, 0.9230038022813688), ({'ONE', 'WHITE'}, 0.9001901140684411), ({'PARTIAL', 'FREE'}, 0.9743346007604563), ({'FREE', 'WHITE'}, 0.9743346007604563), ({'PARTIAL', 'WHITE'}, 0.9771863117870723), ({'ONE', 'PARTIAL', 'FREE'}, 0.9001901140684411), ({'ONE', 'FREE', 'WHITE'}, 0.9001901140684411), ({'ONE', 'PARTIAL', 'WHITE'}, 0.9001901140684411), ({'PARTIAL', 'FREE', 'WHITE'}, 0.9743346007604563), ({'PARTIAL', 'FREE', 'ONE', 'WHITE'}, 0.9001901140684411)]


In [64]:
print("Résultat du chargement de 'titatnic-red.txt', on obtient : ")
Base_titanic = chargeBase('datasets/titanic-red.txt',delimiter=' ')
print(len(Base_titanic),' transactions.')

Résultat du chargement de 'titatnic-red.txt', on obtient : 
741  transactions.


In [65]:
# décommenter les lignes suivantes:

print("Pour la BASE Titanic :")
noms_items(Base_titanic)

print("Exemple: singletons(Base_titanic) rend :")
singletons(Base_titanic)

Pour la BASE Titanic :
Exemple: singletons(Base_titanic) rend :


[{'BergenNorway'},
 {'BroomfieldChelmsfordEngland'},
 {'PetworthSussex'},
 {'Portugal'},
 {'BirkdaleEnglandClevelandOhio'},
 {'SaultStMarieON'},
 {'DeerLodgeMT'},
 {'MoscowBronxNY'},
 {'BelfastNI'},
 {'ElkinsParkPA'},
 {'WestHavenCT'},
 {'ProvoUT'},
 {'SwedenAkeleyMN'},
 {'EnglandAlbionNY'},
 {'MinneapolisMN'},
 {'LondonEastOrangeNJ'},
 {'GoteborgSwedenRockfordIL'},
 {'NiceFrance'},
 {'WestHobokenNJ'},
 {'BrynMawrPA'},
 {'OsloNorwayCameronWI'},
 {'KontiolahtiFinlandDetroitMI'},
 {'ChicagoIL'},
 {'BristolAvonJacksonvilleFL'},
 {'CornwallAkronOH'},
 {'SydneyAustralia'},
 {'England'},
 {'StroodKentEnglandDetroitMI'},
 {'SkaraSwedenRockfordIL'},
 {'BuenosAiresArgentinaNewJerseyNJ'},
 {'SurbitonHillSurrey'},
 {'AustriaHungary'},
 {'SwedenWinnipegMN'},
 {'KarbergSwedenJeromeJunctionAZ'},
 {'HarrisburgPA'},
 {'Argentina'},
 {'KrakorydSwedenBloomingtonIL'},
 {'SyriaYoungstownOH'},
 {'RotherfieldSussexEnglandEssexCoMA'},
 {'LondonBrooklynNY'},
 {'DevonEnglandWichitaKS'},
 {'SeattleWAToledoOH'},

In [26]:
#VERIFICATION
Lres = apriori(Base_titanic,0.2)

# tri sur les valeurs de supports:
#print(sorted(Lres, key=lambda data: data[1]))

# tri sur la taille des itemsets
#print(sorted(Lres, key=lambda data: len(data[0])))

for t in sorted(Lres, key=lambda data: data[1],reverse = True):
    if len(t[0])>1:
        print(t)


({'dead', 'male'}, 0.46963562753036436)
({'Southampton', 'male'}, 0.46288798920377866)
({'Southampton', 'dead'}, 0.41295546558704455)
({'Southampton', 'dead', 'male'}, 0.3684210526315789)
({'female', 'alive'}, 0.32388663967611336)
({'Southampton', '2nd'}, 0.30634278002699056)
({'Southampton', 'alive'}, 0.2982456140350877)
({'Southampton', 'female'}, 0.2483130904183536)
({'1st', 'alive'}, 0.23076923076923078)
({'1st', 'male'}, 0.21997300944669365)
({'Southampton', '1st'}, 0.20917678812415655)
({'male', '2nd'}, 0.20647773279352227)
({'Southampton', 'female', 'alive'}, 0.203778677462888)


In [79]:
Lres = apriori(Base_titanic,0.2)

# tri sur les valeurs de supports:
print(sorted(Lres, key=lambda data: data[1]))

# tri sur la taille des itemsets
#print(sorted(Lres, key=lambda data: len(data[0])))

for t in sorted(Lres, key=lambda data: data[1],reverse = True):
    if len(t[0])>1:
        print(t)

[({'2nd', 'male'}, 0.20647773279352227), ({'Southampton', '1st'}, 0.20917678812415655), ({'1st', 'male'}, 0.21997300944669365), ({'Cherbourg'}, 0.23751686909581646), ({'Southampton', 'female'}, 0.2483130904183536), ({'3rd'}, 0.27125506072874495), ({'Southampton', '2nd'}, 0.30634278002699056), ({'2nd'}, 0.3481781376518219), ({'1st'}, 0.3792172739541161), ({'female'}, 0.38866396761133604), ({'Southampton', 'male'}, 0.46288798920377866), ({'male'}, 0.6099865047233468), ({'Southampton'}, 0.7112010796221323)]
({'Southampton', 'male'}, 0.46288798920377866)
({'Southampton', '2nd'}, 0.30634278002699056)
({'Southampton', 'female'}, 0.2483130904183536)
({'1st', 'male'}, 0.21997300944669365)
({'Southampton', '1st'}, 0.20917678812415655)
({'2nd', 'male'}, 0.20647773279352227)


<font color="RED" size="+1">**[Q]**</font>  Utiliser le programme <code>apriori32</code> pour générer des règles d'association (voir les différentes options en annexe) sur les différentes bases de données fournies. Tester différents seuils (support et confiance). 

<font color="RED" size="+1">**[Q]**</font>   Ajouter les deux mesures d'intérêt vues en TD (lift et RR) et afficher leurs valeurs pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.


<font color="RED" size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt suivantes et afficher leurs valeurs
  pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.
  
Par exemple:

\begin{align}
  \mbox{Interest}(X \longrightarrow Y) & =  \frac{P(X,Y)}{P(X)}P(Y) \nonumber\\
 \mbox{IS}(X \longrightarrow Y) & =  \frac{P(X,Y)}{\sqrt{P(X)P(Y)}} \nonumber
\end{align}

### Utilisation de fp-growth

<font color="RED" size="+1">**[Q]**</font>   Utiliser <code>fpgrowth32</code> pour générer des règles d'association sur les différentes
  bases de données. Comparer avec les résultats obtenus dans la section précédente. En  particulier, comparer les temps d'éxecution.

<font color="RED" size="+1">**[Q]**</font>  En essayant différentes valeurs de seuil, tester la génération de règles d'association intéressantes sur les bases de données fournies.

## Annexes: utilisation des programmes *apriori32* et *fpgrowth32*

La documentation complète de ces programmes est disponible aux URL suivantes:
- pour *apriori*: http://www.borgelt.net/doc/apriori/apriori.html
- pour *fp-growth*: http://www.borgelt.net/doc/fpgrowth/fpgrowth.html


Leur format d'utilisation général est:

        ./<programme> [options] infile [outfile]

(où <<code>programme</code>> est soit *apriori32*, soit *fpgrowth32*).

Les options de base de ces 2 programmes sont:
- sans argument: génération des itemsets fréquents (argument {<code>-ts</code> activé par défaut)
- *tr*: pour obtenir les règles d'associations
- *s*: pour fournir une valeur minimale de support. Le support est ici donné en valeur absolue ($n_{AB}$) et non pas en valeur relative ($\frac{n_{AB}}{n}$).
- *m*: pour fournir un nombre minimum d'items dans un itemset
- *n*: pour fournir un nombre maximum d'items dans un itemset


Par exemple (commandes lancées dans le répertoire père du répertoire *datasets/*):

        ./apriori32 -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out  
        ./fpgrowth32 -trs50m2n5 datasets/mushrooms.txt fichier-resultat.out  
